In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [9]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, parameters = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_bm25, bm25], topics, qrels, 
                                names=['tuned_bm25','orignal_bm25'], save_mode = "overwrite", 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"],  save_dir = "/workspace/src/runs_bm25"))
            
            print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20original_tuned_bm25', '20orignal_bm25'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([tuned_bm25, bm25], topics21, qrels, names=['21original_tuned_bm25', '21orignal_bm25'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            tuned_bm25.to_csv(path_or_buf=f"/workspace/src/runs_bm25/original_tuned_bm25.csv")
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.683724
Best setting is ['BR(BM25) c=0.55', 'BR(BM25) bm25.k_1=2.2', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.463407
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.0', 'BR(BM25) bm25.k_3=2']
           name  ndcg_cut_5  ndcg_cut_25     bpref
0    tuned_bm25    0.493989     0.398293  0.556333
1  orignal_bm25    0.425103     0.345709  0.569233
                    name  ndcg_cut_5  ndcg_cut_25     bpref
0  20original_tuned_bm25    0.334210     0.244781  0.489750
1         20orignal_bm25    0.292792     0.228788  0.511142
                    name  ndcg_cut_5  ndcg_cut_25     bpref
0  21original_tuned_bm25    0.650573     0.548735  0.621585
1         21orignal_bm25    0.554767     0.460291  0.626163


In [11]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, _ = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                metrics
            )
            print(pt.Experiment([tuned_bm25, bm25], topics, qrels, names=[folders[i] + '_tuned_bm25', folders[i] + '_orignal_bm25'], save_mode = "overwrite", eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"],  save_dir = "/workspace/src/runs_bm25"))
            
            print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20' + folders[i] + 'tuned_bm25', '20' + folders[i] + '_bm25'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([tuned_bm25, bm25], topics21, qrels, names=['21' + folders[i] + 'tuned_bm25', '21'+ folders[i] + '_bm25'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            tuned_bm25.to_csv(path_or_buf=f"/workspace/src/runs_bm25/{folders[i]}_tuned_bm25.csv")
            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.580154
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.336166
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=1.6', 'BR(BM25) bm25.k_3=5']
                       name  ndcg_cut_5  ndcg_cut_25     bpref
0    conclusions_tuned_bm25    0.444870     0.373296  0.606832
1  conclusions_orignal_bm25    0.381056     0.330067  0.609588
                      name  ndcg_cut_5  ndcg_cut_25     bpref
0  20conclusionstuned_bm25    0.336044     0.247582  0.491007
1       20conclusions_bm25    0.277659     0.212696  0.491767
                      name  ndcg_cut_5  ndcg_cut_25     bpref
0  21conclusionstuned_bm25    0.551520     0.496497  0.720340
1       21conclusions_bm25    0.482386     0.445090  0.725052

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_ind